In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
hrefs = []

links = ["https://denaihati.com/category/islam",
        "https://denaihati.com/category/isusemasa",
        "https://denaihati.com/category/kisah-kehidupan",
        "https://denaihati.com/category/teknologi",
        "https://denaihati.com/category/travelog",
        "https://denaihati.com/category/kesihatan",
        "https://denaihati.com/category/hiburan",
        "https://denaihati.com/category/sosial-media",
        "https://denaihati.com/category/ekonomi",
        "https://denaihati.com/category/blogging"]


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


In [6]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            if r.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")
    
    block = soup.find_all("h1",attrs = {"class":"elementor-heading-title"})
    
    if block is None:
        return

    for link in soup.find_all("h1",attrs = {"class":"elementor-heading-title"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [7]:
max_worker = 5
hrefs = []

for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "html.parser")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page')[1].replace('/','')) for a_ in pgno])
    print(max_page)
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


https://denaihati.com/category/islam
97


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:58<00:00,  5.90s/it]


https://denaihati.com/category/isusemasa
155


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [03:24<00:00,  6.61s/it]


https://denaihati.com/category/kisah-kehidupan
48


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.34s/it]


https://denaihati.com/category/ekonomi
10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:11<00:00,  5.78s/it]


https://denaihati.com/category/blogging
72


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [01:41<00:00,  6.79s/it]


In [9]:
hrefs = list(set(hrefs))

In [10]:
len(hrefs)

3508

In [11]:
with open(f'denaihati-link.json', 'a') as f:
    json.dump(hrefs, f)

In [4]:
with open('denaihati-link.json') as fopen:
    hrefs = json.load(fopen)

In [6]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('h1', class_="elementor-heading-title").text
        h = soup.find('div', attrs = {'class':"elementor-element"})
        p = [p.text for p in h.find_all(['p','h1','h2','h3','h4']) if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    comment_list = soup.find('div', attrs={"class": "comments-area"})
    comments = []
    if comment_list:
        c = comment_list.find_all('div', attrs={"class": "comment-content"})
        for j in c:
            comments.extend([c_.text for c_ in j.find_all("p") if len(c_.text) > 1])
            
        if c is None:
            comments = None


    
    data = {'url': x, 'headline': headline, 'content': p, 'comment': comments}
    
    return data

In [7]:
process_url("https://denaihati.com/12-golongan-manusia-yang-didoakan-kebaikan-oleh-malaikat")

{'url': 'https://denaihati.com/12-golongan-manusia-yang-didoakan-kebaikan-oleh-malaikat',
 'headline': '12 Golongan Manusia Yang Didoakan Kebaikan Oleh Malaikat',
 'content': ['12 Golongan Manusia Yang Didoakan Kebaikan Oleh Malaikat',
  'Jika artikel sebelum ini ada menceritakan mengenai 3 golongan yang didoakan keburukan oleh malaikat, kali ini saya nak senaraikan 12 golongan yang didoakan kebaikan oleh malaikat.',
  'Baca di sini : Golongan Yang Didoakan Keburukan Oleh Malaikat, Nauzubillah',
  'Berusahalah untuk menjadi golongan-golongan ini supaya hidup menjadi lebih harmoni dan tenang.',
  '1) Orang yang tidur dalam keadaan bersuciRasulullah SAW bersabda, “Barangsiapa yang tidur dalam keadaan suci, maka malaikat akan bersamanya di dalam pakaiannya. Dia tidak akan bangun hingga malaikat berdoa ‘Ya Allah, ampunilah hambamu si fulan kerana tidur dalam keadaan suci”.',
  '(Imam Ibnu Hibban meriwayatkan dari Abdullah bin Umar ra., hadits ini dishahihkan oleh Syaikh Al Albani dalam Sha

In [8]:
max_workers = 5

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('denaihati.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3508/3508 [21:38<00:00,  2.70it/s]
